# **Template OP on salish**

In [1]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_sink import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift
from OP_functions import *

In [2]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [3]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
batt=xr.open_dataset(paths['mask'],decode_times=False)
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))

In [4]:
clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10)

## Definitions

In [10]:
start = datetime(2018, 12, 10) #Start date
# Set Time length [days] and timestep [seconds]
length = 25
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = len(clat) # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = list(np.zeros(len(clat)))
#dmin = [0,0,0,0,0,70] #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [16]:
N

841

In [11]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1370)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    sediment = Variable('sediment', initial = 0)


In [12]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
#clon, clat = [-123.901172,-125.155849,-123.207648,-122.427508,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,47.622403,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [13]:
#Set start date time and the name of the output file
name = 'uniformL' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/uniformL20181210_1n_20190104_1n.nc


## Simulation

In [14]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [15]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)

pset.execute(AdvectionRK4_3D + k_sink + k_waves,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_drift ==> /tmp/parcels-2894/liba04591201e137850da9a84642993abdd_0.so


Particle 829 lost !! [5400.0, 2.1164774376831916, 50.444054915644315, -126.0136770622564]
Particle 837 lost !! [5400.0, 2.979672508104655, 50.461517184542465, -126.10192331052394]


INFO: Temporary output files are stored in /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-AKSCFKUH.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-AKSCFKUH" to convert these to a NetCDF file during the run.
  4% (95400.0 of 2160000.0) |            | Elapsed Time: 0:01:24 ETA:   0:48:27

Particle 824 lost !! [101700.0, 12.831506200415344, 50.42367653008263, -125.99902337757364]


100% (2160000.0 of 2160000.0) |##########| Elapsed Time: 0:31:35 Time:  0:31:35
/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
